# Imports, install and mount

<!--  -->

In [2]:
# ! pip install cuda
# ! pip install torch_geometric
# ! pip install nxontology
# ! pip install tensordict
# ! pip install pandas
# ! pip install tensorflow
# ! pip install scipy
# ! pip install matplotlib

# ! pip3 install torch==2.0.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch_geometric
from torch_geometric.nn import ComplEx
from torch_geometric.data import Data
import pandas as pd
from tqdm import tqdm
from torch_geometric.loader import DataLoader

import wandb

import pickle

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# Settings


In [7]:
# ComplEx embeddings :

hidden_channels = 220
batch_size = 4096
epochs = 1000

params_save_name = f"PARAMS_ComplEx_HC_6_times_{hidden_channels}_on_full_Os_GO"
params_save_path = "/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/"+params_save_name

datasets_save_path = '/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/dataset_BS_'+str(batch_size)
val_path = datasets_save_path + 'VAL' +  '.pickle'
test_path = datasets_save_path + 'TEST' +  '.pickle'
train_path = datasets_save_path + 'TRAIN' +  '.pickle'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(device)

# wandb.init(
#     settings=wandb.Settings(start_method="fork"),
#     # set the wandb project where this run will be logged
#     project="ComplEx on Os_to_GO_iric",
    
#     # track hyperparameters and run metadata
#     config={
#     "architecture": "ComplEx",
#     "dataset": "genes_to_phenotypes_iric.tsv",
#     "epochs": epochs,
#     'hidden_channels' : hidden_channels,
#     'batch_size' : batch_size
#     }
# )

cpu


# DATAS

In [8]:
def save_dataset(dataset: torch_geometric.data.data.Data, save_path: str):
    with open(save_path, 'wb') as f:
        pickle.dump(dataset.to_dict(), f, pickle.HIGHEST_PROTOCOL)

def load_dataset(dataset_path: str # Should lead to a  '.pickle' file
                 )-> torch_geometric.data.data.Data:
    with open(dataset_path, 'rb') as f:
        return torch_geometric.data.data.Data.from_dict(pickle.load(f))
    torch.load with map_location=torch.device('cpu')
    
load_dataset('/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/dataset_TEST.pickle')

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

## Building init vars for Data :

In [15]:
# # Initial nodes states :
# x = torch.ones(len(entity_set), 1)  # Chaque nœud a 1 pour état initial
# print('X : \n',x)

# Edges index
heads = list(iric['mapped_subject'])
tails = list(iric['mapped_object'])
edge_index = torch.tensor([heads,tails], dtype=torch.long)
print('\nEDGE INDEX : \n',edge_index)

# edges states
edge_attributes = torch.tensor(iric['mapped_predicate'])
print('\nEDGES ATTRIBUTES : \n',edge_attributes)

iric_pyg = Data(
                num_nodes = len(entity_set),
                edge_index = edge_index,
                edge_attr = edge_attributes
                )

print("\nDataset looks valid ? \n",iric_pyg.validate(raise_on_error=True))


EDGE INDEX : 
 tensor([[24385, 24385, 24385,  ..., 13194,  3131,  3131],
        [26614, 25106,  6930,  ..., 26477,  6836, 11218]])

EDGES ATTRIBUTES : 
 tensor([0, 0, 0,  ..., 0, 0, 0])

Dataset looks valid ? 
 True


## Setting up datas and model


## Splitting dataset

In [16]:
from torch_geometric.transforms import RandomLinkSplit

transform = RandomLinkSplit(
                            num_val = 0.1,
                            num_test = 0.1,
                            is_undirected=False,
                            add_negative_train_samples=False,
                            )

train_data, val_data, test_data = transform(iric_pyg)

# print(f"Hole Dataset :\n {iric_pyg}\n\nTrain:\n{train_data}\n\nTest :\n{test_data}\n\nValidation :\n{val_data}")
# # Il ne faut pas regarder num_edges parce que RLS cache les arêtes mais ne les sort pas du graph.
# # print(f"Number of edges in datasets : \n  Hole Dataset : {iric_pyg.num_edges}\n\n  Train: {train_data.num_edges}\n\n  Test : {test_data.num_edges}\n\n  Validation : {val_data.num_edges}")
# print(f"Number of edges in datasets : \n  Train: {list(train_data.edge_label.size())[0]}\n\n  Test : {list(test_data.edge_label.size())[0]}\n\n  Validation : {list(val_data.edge_label.size())[0]}")

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

print(type(train_data))

# print('\n\n',train_data.num_nodes)
# print(train_data.num_edge_types)
# print(train_data.__dict__)
# print(train_data.edge_index[0].size())
# print(train_data.edge_index[1].size())
# print(train_data.edge_attr.size())
# print(train_data.edge_attr)
# print(train_data.num_nodes)
# print(train_data.edge_index.size()[1])

<class 'torch_geometric.data.data.Data'>


In [17]:
def save_dataset(dataset: torch_geometric.data.data.Data, save_path: str):
    with open(save_path, 'wb') as f:
        pickle.dump(dataset.to_dict(), f, pickle.HIGHEST_PROTOCOL)

def load_dataset(dataset_path: str # Should lead to a  '.pickle' file
                 )-> torch_geometric.data.data.Data:
    with open(dataset_path, 'rb') as f:
        return torch_geometric.data.data.Data.from_dict(pickle.load(f))
    
save_dataset(dataset=val_data, save_path=val_path)
save_dataset(dataset=test_data, save_path=test_path)
save_dataset(dataset=train_data, save_path=train_path)


# MODELS


## Iniating models and loaders

In [18]:
# Initiating models

complex_model = ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
).to(device)

# Initiaing loader
head_index = train_data.edge_index[0]
tail_index = train_data.edge_index[1]
rel_type = train_data.edge_attr

loader = complex_model.loader(
    head_index = head_index,
    tail_index = tail_index,
    rel_type = rel_type,
    batch_size=batch_size,
    shuffle=True,
)

# initiating optimizers
complex_optimizer = optim.Adam(complex_model.parameters())

print(batch_size)

4096


## Train and test functions

In [19]:
@torch.no_grad()
def test(data, model):
    model.eval()
    return model.test(
        head_index=test_data.edge_index[0],
        tail_index=test_data.edge_index[1],
        rel_type=test_data.edge_attr,
        batch_size=batch_size,
        k=10,
    )

def train(model, optimizer):
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples

def plot_loss(loss_list):
    plt.plot(loss_list, label='Loss')
    plt.title('Evolution des Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_loss_log(loss_list):
    plt.semilogy(loss_list, label='Loss')
    plt.title('Evolution des Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def running_mean(list,
                 half_window: int # Number of elements that the function will consider
                                  # ahead and behind the position X to calculate running mean at X.
                 ):
    running_means = []

    for i in range(0,len(list)):

        left_bound = max(0,i-half_window)
        right_bound = min(len(list)-1, i + half_window)
        sublist = list[left_bound:right_bound+1]
        running_means.append(sum(sublist)/len(sublist))

    return running_means

## Train and test

In [20]:
torch.set_grad_enabled(True)

complex_model.reset_parameters()
complex_model.to(device)

losses = []
for epoch in range(0, epochs+1):
    loss = train(model=complex_model, optimizer=complex_optimizer)
    losses.append(loss)
    wandb.log({"loss": loss})
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

    if epoch % epochs%1000 == 0:
        rank, mrr, hits = test(val_data, model=complex_model)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}',
              f'Val MRR: {mrr:.4f}, Val Hits@10: {hits:.4f}')
        wandb.log({"Val Mean Rank" : rank, "Val MRR" : mrr, "hits@10": hits})


torch.set_grad_enabled(False)

Epoch: 000, Loss: 0.6931


100%|██████████| 152324/152324 [15:32<00:00, 163.37it/s]


Epoch: 000, Val Mean Rank: 8673.97 Val MRR: 0.0084, Val Hits@10: 0.0152
Epoch: 001, Loss: 0.6931
Epoch: 002, Loss: 0.6919
Epoch: 003, Loss: 0.6790
Epoch: 004, Loss: 0.6337
Epoch: 005, Loss: 0.5650
Epoch: 006, Loss: 0.5067
Epoch: 007, Loss: 0.4716
Epoch: 008, Loss: 0.4505
Epoch: 009, Loss: 0.4338
Epoch: 010, Loss: 0.4214
Epoch: 011, Loss: 0.4078
Epoch: 012, Loss: 0.3963
Epoch: 013, Loss: 0.3798
Epoch: 014, Loss: 0.3634
Epoch: 015, Loss: 0.3454
Epoch: 016, Loss: 0.3260
Epoch: 017, Loss: 0.3041
Epoch: 018, Loss: 0.2823
Epoch: 019, Loss: 0.2609
Epoch: 020, Loss: 0.2403
Epoch: 021, Loss: 0.2203
Epoch: 022, Loss: 0.2039
Epoch: 023, Loss: 0.1869
Epoch: 024, Loss: 0.1727
Epoch: 025, Loss: 0.1618
Epoch: 026, Loss: 0.1496
Epoch: 027, Loss: 0.1387
Epoch: 028, Loss: 0.1305
Epoch: 029, Loss: 0.1216
Epoch: 030, Loss: 0.1159
Epoch: 031, Loss: 0.1104
Epoch: 032, Loss: 0.1032
Epoch: 033, Loss: 0.0997
Epoch: 034, Loss: 0.0942
Epoch: 035, Loss: 0.0915
Epoch: 036, Loss: 0.0873
Epoch: 037, Loss: 0.0825
Epo

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
wandb.finish()
print("WandB finished.")

torch.save(complex_model.state_dict(), params_save_path)
print("Model saved at", params_save_path)

Val MRR,▁█
Val Mean Rank,█▁
hits@10,▁█
loss,█▅▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val MRR,0.16405
Val Mean Rank,230.30151
hits@10,0.36778
loss,0.06995


WandB finished.
Model saved at /home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/PARAMS_ComplEx_HC_6_times_5_on_full_Os_GO
